In [11]:
import os
import json
import time
import urllib
import requests
import datetime

headers = {'Content-Type': 'multipart/form-data'}

In [12]:
def read_json(filename):
    with open(filename) as f:
        return json.load(f)
    
def write_json(data, filename):
    with open(filename, 'w') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)
        
def upload_file(file, name, type, multiselect=None):
    if len(file) > 100:
        raise Exception("File names must not exceed 100 characters!")
    files = {
        "file": (file, open(file, 'rb'), type)
    }
    data = {
        'name': name,
        'multiselect': multiselect 
    }
    request = requests.post("http://gerbil-qa.cs.uni-paderborn.de:8080/gerbil/file/upload", data=data, files=files)
    response = request.json()
    if request.status_code == 200:
        return response
    else: 
        raise Exception(f"file upload not successful: {request.status_code}: {request.content}")

In [3]:
url_postfix = """experimentData={{"type":"QA","matching":"STRONG_ENTITY_MATCH","annotator":["{annotator}"],"dataset":["{dataset}"],"answerFiles":[],"questionLanguage":"{lang}"}}"""
uploaded_dataset_prefix = "NIFDS"

data_folder = "translated_data"
files = [os.path.join(data_folder, f) for f in os.listdir("translated_data")]

dataset_dict = {
    'English': 'en',
    # 'German': 'de',
    # 'Russian': 'ru',
    # 'French': 'fr'
}

annotators = [
    'QAnswer',
    'Qanary_1',
    'Qanary_2',
    'Qanary_3',
    'Qanary_4',
    'Qanary_5',
]

In [4]:
for file in files:
    for annotator_ in annotators:
        annotator = "NIFWS_{0}(http://porque.cs.upb.de:40123/{1}/gerbil)".format(annotator_, annotator_.lower())
        for src_lang, lang_code in zip(dataset_dict.keys(), dataset_dict.values()):
            gold_standard_file = file
            gold_standard_name = file.split(".")[0]
            upload_file(gold_standard_file, gold_standard_name, 'application/json')

            execute_url = "http://gerbil-qa.aksw.org/gerbil/execute?" + url_postfix.format(
                annotator=annotator,
                dataset=f"{uploaded_dataset_prefix}_{gold_standard_name}({gold_standard_file})",
                lang=lang_code
            )

            is_ok = False
            while not is_ok:
                try:
                    if len(requests.get("http://gerbil-qa.aksw.org/gerbil/running").text) < 5000:
                        response = requests.get(execute_url)
                        experiment_url = "http://gerbil-qa.aksw.org/gerbil/experiment?id={0}".format(response.text)
                        print(annotator, src_lang, experiment_url)
                        with open("experiments.log", 'a') as f:
                            c_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                            f.write(f"{annotator}\t{gold_standard_name}\t{src_lang}\t{experiment_url}\t{c_time}\n")

                        is_ok = True
                    else:
                        time.sleep(60)
                    
                except:
                    time.sleep(60)

NIFWS_Qanary_5(http://porque.cs.upb.de:40123/qanary_5/gerbil) English http://gerbil-qa.aksw.org/gerbil/experiment?id=202209280000
NIFWS_Qanary_5(http://porque.cs.upb.de:40123/qanary_5/gerbil) English http://gerbil-qa.aksw.org/gerbil/experiment?id=202209280001


# QALD 9 Plus (DBpedia)

## Test


### QAnswer

* English http://gerbil-qa.aksw.org/gerbil/experiment?id=202209230009
* German http://gerbil-qa.aksw.org/gerbil/experiment?id=202209250000
* Russian http://gerbil-qa.aksw.org/gerbil/experiment?id=202209250001
* French http://gerbil-qa.aksw.org/gerbil/experiment?id=202209250002

## Train

### QAnswer

* English http://gerbil-qa.aksw.org/gerbil/experiment?id=202209240013
* German http://gerbil-qa.aksw.org/gerbil/experiment?id=202209240014
* Russian http://gerbil-qa.aksw.org/gerbil/experiment?id=202209240015
* French http://gerbil-qa.aksw.org/gerbil/experiment?id=202209240016

# QALD 10
